In [1]:
# !pip install grad-cam

In [2]:
DATA_DIR = './CAM-data'

import warnings
warnings.filterwarnings('ignore')
from torchvision import models
import numpy as np
import cv2
import requests
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    deprocess_image, \
    preprocess_image
from PIL import Image

# img = cv2.resize(img, (224, 224))
# img = np.float32(img) / 255
# input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [3]:
from pytorch_grad_cam import AblationCAM, GradCAM, GradCAMPlusPlus, ScoreCAM, EigenCAM
CAM_MAPPING = {
    'GradCAM': GradCAM,
    'GradCAMPlusPlus': GradCAMPlusPlus,
    'EigenCAM': EigenCAM,
    'AblationCAM': AblationCAM,
    'ScoreCAM': ScoreCAM,
}


MODEL_TARGET_LAYER_EXTRACTOR = {
    "DenseNet201": lambda x: x.features[-1],
    "ConvNeXt_Large": lambda x: x.features[-1],
    "Inception_V3": lambda x: x.Mixed_7c,
    "ResNet152": lambda x: x.layer4,
    "MobileNet_V2": lambda x: x.features[-1],
    "EfficientNet_B7": lambda x: x.features[-2],
    "VGG19": lambda x: x.features[-1],
}

WEIGHT = 'IMAGENET1K_V1'


In [4]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torchvision.transforms as transforms

class ImageNet(Dataset):
    def __init__(self, data_dir: str = DATA_DIR, return_labels: bool = False):
        
        self.label_mapping, self.reversed_mapping_names = self.split_labels()
        self.data_dir = data_dir
        
        self.metadata = pd.read_csv('./file_paths.csv')
        self.file_paths = self.metadata['path'].apply(
            lambda x: (
                self.data_dir
                + '/' + x + '.JPEG'
            )
        )

        self.return_labels = return_labels

    @staticmethod
    def split_labels():
        with open('./LOC_synset_mapping.txt', 'r') as f:
            txt = f.read()
        lines = txt.split('\n')[:-1] # ignore last line

        mapping = dict()
        reversed_mapping_name = dict()
        for idx, line in enumerate(lines):
            pos = line.find(' ')
            mapping[line[:pos]] = idx
            reversed_mapping_name[idx] = line[pos+1:]

        return mapping, reversed_mapping_name

        
    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = np.array(Image.open(img_path))
        image = cv2.resize(image, (224, 224))
        image = np.float32(image) / 255.
        if len(image.shape) == 2:
            image = np.stack([image] * 3, axis=-1)
        input_tensor = preprocess_image(image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        
        label_str = self.metadata['path'][idx].split('_')[0]
        label_idx = self.label_mapping[label_str]
        

        if self.return_labels:
            return input_tensor, image, label_idx
        return input_tensor, image

In [5]:
from torchvision import models
from tqdm import tqdm
from torch import nn
import torch
from pathlib import Path
import os

FAILED = []

def generate_cam(model_name, weight, cam_type, data_dir, device: str = 'cuda', save_dir: str = './'):
    model = models.get_model(model_name, pretrained=weight)
    model.to(device=device)
    model.eval()
    
    dataset = ImageNet(data_dir, return_labels=True)
    target_layers = [MODEL_TARGET_LAYER_EXTRACTOR[model_name](model)]
    cam = CAM_MAPPING[cam_type](model=model, target_layers=target_layers)

    model_save_dir = Path(save_dir) / f"{model_name}" / f"{cam_type}"
    os.makedirs(model_save_dir, exist_ok=True)

    for idx in tqdm(range(dataset.__len__())):
        if os.path.exists(model_save_dir / f"{idx}.png"):
            continue
        (input_tensor, img, gt_label_idx)  = dataset.__getitem__(idx)
        pred_label_idx = nn.Softmax(dim=1)(model(input_tensor.to(device))).argmax()

        pred_idx = pred_label_idx.to('cpu').item()
        gt_idx = gt_label_idx

        if pred_idx != gt_idx:
            print(f"FAILED at: image {idx} of {model_name} with {cam_type}."
                  f"Predicted: {dataset.reversed_mapping_names[pred_idx]}, "
                  f"Ground Truth: {dataset.reversed_mapping_names[gt_idx]}")
            FAILED.append((model_name, idx))
        
        # tmp_img = tmp_img.permute(1, 2, 0)
        target_categories = [ClassifierOutputTarget(gt_label_idx)]
        grayscale_cam = cam(input_tensor=input_tensor, targets=target_categories)

        visualization = show_cam_on_image(img, grayscale_cam[0, :], use_rgb=True)
        visualization = Image.fromarray(visualization)

        visualization.save(model_save_dir / f"{idx}.png")

for model_name in MODEL_TARGET_LAYER_EXTRACTOR.keys():
    for technique in CAM_MAPPING.keys():
        generate_cam(model_name, WEIGHT, technique, DATA_DIR, 'cuda', './')

 37%|███▋      | 372/1000 [00:11<00:18, 34.47it/s]

FAILED at: image 367 of ConvNeXt_Large with GradCAM.Predicted: washbasin, handbasin, washbowl, lavabo, wash-hand basin, Ground Truth: pedestal, plinth, footstall


 40%|████      | 400/1000 [00:12<00:17, 34.30it/s]

FAILED at: image 393 of ConvNeXt_Large with GradCAM.Predicted: Australian terrier, Ground Truth: Norwich terrier


 44%|████▎     | 436/1000 [00:13<00:16, 34.53it/s]

...

FAILED at: image 261 of EfficientNet_B7 with ScoreCAM.Predicted: hay, Ground Truth: swab, swob, mop


 28%|██▊       | 279/1000 [03:54<10:03,  1.19it/s]

FAILED at: image 279 of EfficientNet_B7 with ScoreCAM.Predicted: brass, memorial tablet, plaque, Ground Truth: doormat, welcome mat


 29%|██▉       | 292/1000 [04:05<09:55,  1.19it/s]


KeyboardInterrupt: 

In [ ]:
# ImageNet(DATA_DIR).file_paths[39]

In [ ]:
# mapping = ImageNet(DATA_DIR).label_mapping

In [ ]:
# #fine key with value
# def get_key(val):
#     for key, value in mapping.items():
#         if val == value:
#             return key
        
# get_key(39)

In [ ]:
# import warnings
# from regex import B
# warnings.filterwarnings('ignore')
# from torchvision import models
# import numpy as np
# import cv2
# import requests
# from pytorch_grad_cam import GradCAM, GradCAMPlusPlus
# from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
# from pytorch_grad_cam.utils.image import show_cam_on_image, \
#     deprocess_image, \
#     preprocess_image
# from PIL import Image

# model =  models.get_model("ConvNeXt_Large", pretrained="IMAGENET1k_V1")
# model.eval()
# image_url = "https://th.bing.com/th/id/R.94b33a074b9ceeb27b1c7fba0f66db74?rik=wN27mvigyFlXGg&riu=http%3a%2f%2fimages5.fanpop.com%2fimage%2fphotos%2f31400000%2fBear-Wallpaper-bears-31446777-1600-1200.jpg&ehk=oD0JPpRVTZZ6yizZtGQtnsBGK2pAap2xv3sU3A4bIMc%3d&risl=&pid=ImgRaw&r=0"
# img = np.array(Image.open('./CAM-data/n01440764_11816.JPEG'))
# img = cv2.resize(img, (224, 224))
# img = np.float32(img) / 255
# input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# print(input_tensor.max(), input_tensor.min())
# # The target for the CAM is the Bear category.
# # As usual for classication, the target is the logit output
# # before softmax, for that category.
# targets = [ClassifierOutputTarget(0)]
# target_layers = [model.features[-9]]
# with GradCAMPlusPlus(model=model, target_layers=target_layers) as cam:
#     grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
#     print(img.shape, type(img))
#     print(grayscale_cams[0, :].shape, type(grayscale_cams[0, :]))
#     cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)
# cam = np.uint8(255*grayscale_cams[0, :])
# cam = cv2.merge([cam, cam, cam])
# images = np.hstack((np.uint8(255*img), cam , cam_image))
# Image.fromarray(images)

In [ ]:
# ImageNet(DATA_DIR).label_mapping['n01440764']

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')
# from torchvision import models
# import numpy as np
# import cv2
# import requests
# from pytorch_grad_cam import GradCAM, AblationCAM, ScoreCAM, EigenCAM
# from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
# from pytorch_grad_cam.utils.image import show_cam_on_image, \
#     deprocess_image, \
#     preprocess_image
# from PIL import Image

# # model_name = MODELS[2]
# model_name = "ConvNeXt_Large"
# print(model_name)
# model = models.get_model(model_name, pretrained=WEIGHT)
# model.to("cuda")
# model.eval()
# image_url = "https://th.bing.com/th/id/R.94b33a074b9ceeb27b1c7fba0f66db74?rik=wN27mvigyFlXGg&riu=http%3a%2f%2fimages5.fanpop.com%2fimage%2fphotos%2f31400000%2fBear-Wallpaper-bears-31446777-1600-1200.jpg&ehk=oD0JPpRVTZZ6yizZtGQtnsBGK2pAap2xv3sU3A4bIMc%3d&risl=&pid=ImgRaw&r=0"
# img = np.array(Image.open(requests.get(image_url, stream=True).raw))
# img = cv2.resize(img, (224, 224))
# img = np.float32(img) / 255
# input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# # The target for the CAM is the Bear category.
# # As usual for classication, the target is the logit output
# # before softmax, for that category.
# targets = [ClassifierOutputTarget(295)]
# # target_layers = [model.features[-1]]
# # target_layers = [model.Mixed_7c]
# target_layers = [model.features[-1]]
# # target_layers = [model.layer4]

# with EigenCAM(model=model, target_layers=target_layers) as cam:
#     grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
#     print(img.shape, grayscale_cams[0, :].shape)
#     cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)
# cam = np.uint8(255*grayscale_cams[0, :])
# print(cam.shape)
# cam = cv2.merge([cam, cam, cam])
# images = np.hstack((np.uint8(255*img), cam , cam_image))
# Image.fromarray(images)

In [ ]:
# model.features[-1]